<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/llm_graph_transformer_in_depth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
# !pip install --quiet neo4j langchain-neo4j langchain-experimental langchain-openai

## LLM Graph Transformer
The LLM Graph Transformer was designed to provide a flexible framework for building graphs using any LLM. With so many different providers and models available, this task is far from simple. Fortunately, LangChain steps in to handle much of the standardization process. As for the LLM Graph Transformer itself, it's like two cats stacked in a trench coat -with the ability to operate in two completely independent modes.

![image](https://cdn-images-1.medium.com/max/2400/1*aCSCXuvrOB90jRQ0mNZtSA.png)

The LLM Graph Transformer operates in two distinct modes, each designed to generate graphs from documents using an LLM in different scenarios.

* **Tool-Based Mode (Default)**: When the LLM supports structured output or function calling, this mode leverages the LLM's built-in with_structured_outputto use tools. The tool specification defines the output format, ensuring that entities and relationships are extracted in a structured, predefined manner. This is depicted on the left side of the image, where code for the Node and Relationship classes is shown.
* **Prompt-Based Mode (Fallback)** : In situations where the LLM doesn't support tools or function calls, the LLM Graph Transformer falls back to a purely prompt-driven approach. This mode uses few-shot prompting to define the output format, guiding the LLM to extract entities and relationships in a text-based manner. The results are then parsed through a custom function, which converts the LLM's output into a JSON format. This JSON is used to populate nodes and relationships, just as in the tool-based mode, but here the LLM is guided entirely by prompting rather than structured tools. This is shown on the right side of the image, where an example prompt and resulting JSON output are provided.

These two modes ensure that the LLM Graph Transformer is adaptable to different LLMs, allowing it to build graphs either directly using tools or by parsing output from a text-based prompt.

_Note that you can use prompt-based extraction even with models that support tools/functions by setting the attribute ignore_tools_usage=True._

## Tool-based extraction
We initially chose a tool-based approach for extraction since it minimized the need for extensive prompt engineering and custom parsing functions. In LangChain, the with_structured_output method allows you to extract information using tools or functions, with output defined either through a JSON structure or a Pydantic object. Personally, I find Pydantic objects clearer, so we opted for that.
We start by defining a Node class.
```
class Node(BaseNode):
    id: str = Field(..., description="Name or human-readable unique identifier")
    label: str = Field(..., description=f"Available options are {enum_values}")
    properties: Optional[List[Property]]
```
Each node has an id, a label, and optional properties. For brevity, I haven't included full descriptions here. Describing ids as human-readable unique identifier is important since some LLMs tend to understand ID properties in more traditional way like random strings or incremental integers. Instead we want the name of entities to be used as id property. We also limit the available label types by simply listing them in the labeldescription. Additionally, LLMs like OpenAI's, support an enum parameter, which we also use.
Next, we take a look at the Relationship class
```
class Relationship(BaseRelationship):
    source_node_id: str
    source_node_label: str = Field(..., description=f"Available options are {enum_values}")
    target_node_id: str
    target_node_label: str = Field(..., description=f"Available options are {enum_values}")
    type: str = Field(..., description=f"Available options are {enum_values}")
    properties: Optional[List[Property]]
```
This is the second iteration of the Relationship class. Initially, we used a nested Node object for the source and target nodes, but we quickly found that nested objects reduced the accuracy and quality of the extraction process. So, we decided to flatten the source and target nodes into separate fields-for example, source_node_id and source_node_label, along with target_node_id and target_node_label. Additionally, we define the allowed values in the descriptions for node labels and relationship types to ensure the LLMs adhere to the specified graph schema.
The tool-based extraction approach enables us to define properties for both nodes and relationships. Below is the class we used to define them.
class Property(BaseModel):
    """A single property consisting of key and value"""
    key: str = Field(..., description=f"Available options are {enum_values}")
    value: str
Each Property is defined as a key-value pair. While this approach is flexible, it has its limitations. For instance, we can't provide a unique description for each property, nor can we specify certain properties as mandatory while others optional, so all properties are defined as optional. Additionally, properties aren't defined individually for each node or relationship type but are instead shared across all of them.
We've also implemented a detailed system prompt to help guide the extraction. In my experience, though, the function and argument descriptions tend to have a greater impact than the system message.
Unfortunately, at the moment, there is no simple way to customize function or argument descriptions in LLM Graph Transformer.
## Prompt-based extraction
Since only a few commercial LLMs and LLaMA 3 support native tools, we implemented a fallback for models without tool support. You can also set ignore_tool_usage=True to switch to a prompt-based approach even when using a model that supports tools.
Most of the prompt engineering and examples for the prompt-based approach were contributed by Geraldus Wilsen.
With the prompt-based approach, we have to define the output structure directly in the prompt. You can find the whole prompt here. In this blog post, we'll just do a high-level overview. We start by defining the system prompt.
```
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph. Your task is to identify the entities and relations specified in the user prompt from a given text and produce the output in JSON format. This output should be a list of JSON objects, with each object containing the following keys:

- **"head"**: The text of the extracted entity, which must match one of the types specified in the user prompt.
- **"head_type"**: The type of the extracted head entity, selected from the specified list of types.
- **"relation"**: The type of relation between the "head" and the "tail," chosen from the list of allowed relations.
- **"tail"**: The text of the entity representing the tail of the relation.
- **"tail_type"**: The type of the tail entity, also selected from the provided list of types.

Extract as many entities and relationships as possible.

**Entity Consistency**: Ensure consistency in entity representation. If an entity, like "John Doe," appears multiple times in the text under different names or pronouns (e.g., "Joe," "he"), use the most complete identifier consistently. This consistency is essential for creating a coherent and easily understandable knowledge graph.

**Important Notes**:
- Do not add any extra explanations or text.
```
In the prompt-based approach, a key difference is that we ask the LLM to extract only relationships, not individual nodes. This means we won't have any isolated nodes, unlike with the tool-based approach. Additionally, because models lacking native tool support typically perform worse, we do not allow extraction any properties - whether for nodes or relationships, to keep the extraction output simpler.
Next, we add a couple of few-shot examples to the model.
```
examples = [
    {
        "text": (
            "Adam is a software engineer in Microsoft since 2009, "
            "and last year he got an award as the Best Talent"
        ),
        "head": "Adam",
        "head_type": "Person",
        "relation": "WORKS_FOR",
        "tail": "Microsoft",
        "tail_type": "Company",
    },
    {
        "text": (
            "Adam is a software engineer in Microsoft since 2009, "
            "and last year he got an award as the Best Talent"
        ),
        "head": "Adam",
        "head_type": "Person",
        "relation": "HAS_AWARD",
        "tail": "Best Talent",
        "tail_type": "Award",
    },
...
]
```
In this approach, there's currently no support for adding custom few-shot examples or extra instructions. The only way to customize is by modifying the entire prompt through the promptattribute. Expanding customization options is something we're actively considering.
Next, we'll take a look at defining the graph schema.
## Defining the graph schema
When using the LLM Graph Transformer for information extraction, defining a graph schema is essential for guiding the model to build meaningful and structured knowledge representations. A well-defined graph schema specifies the types of nodes and relationships to be extracted, along with any attributes associated with each. This schema serves as a blueprint, ensuring that the LLM consistently extracts relevant information in a way that aligns with the desired knowledge graph structure.

In this blog post, we'll use the opening paragraph of Marie Curie's Wikipedia page for testing with an added sentence at the end about Robin Williams.

In [98]:
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader

loader = TextLoader("albert_einstein.txt")
documents = loader.load()

# Building knowledge graphs with LLM Graph Transformer
## A deep dive into LangChain's implementation of graph construction with LLMs

Creating graphs from text is incredibly exciting, but definitely challenging. Essentially, it's about converting unstructured text into structured data. While this approach has been around for some time, it gained significant traction with the advent of Large Language Models (LLMs), bringing it more into the mainstream.

![image](https://cdn-images-1.medium.com/max/1600/0*qgT2hBiA3DA1Y3qu.png)

In the image above, you can see how information extraction transforms raw text into a knowledge graph. On the left, multiple documents show unstructured sentences about individuals and their relationships with companies. On the right, this same information is represented as a graph of entities and their connections, showing who worked at or founded various organizations.
But why would you want to extract structured information from text and represent it as a graph? One key reason is to power retrieval-augmented generation (RAG) applications. While using text embedding models over unstructured text is an useful approach, it can fall short when it comes to answering complex, multi-hop questions that require understanding connections across multiple entities or question where structured operations like filtering, sorting, and aggregation is required. By extracting structured information from text and constructing knowledge graphs, you not only organize data more effectively but also create a powerful framework for understanding complex relationships between entities. This structured approach makes it much easier to retrieve and leverage specific information, expanding the types of questions you can answer while providing greater accuracy.

Around a year ago, I began experimenting with building graphs using LLMs, and due to the growing interest, we decided to integrate this capability into LangChain as the LLM Graph Transformer. Over the past year, we've gained valuable insights and introduced new features, which we'll be showcasing in this blog post.


## Setting up Neo4j environment

We will use Neo4j as the underlying graph store, which comes with out-of-the box graph visualizations. The easiest way to get started is to use a free instance of Neo4j Aura, which offers cloud instances of the Neo4j database. Alternatively, you can set up a local instance of the Neo4j database by downloading the Neo4j Desktop application and creating a local database instance.

We'll also be using GPT-4o in all examples.

In [99]:
from langchain_neo4j import Neo4jGraph
from dotenv import load_dotenv

import os

load_dotenv()

password = os.getenv("NEO4J_PASSWORD")
graph = Neo4jGraph(
    url="neo4j://127.0.0.1:7687",
    username="neo4j",
    password=password,
    refresh_schema=False,
)


def clean_graph():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    graph.query(query)

To start, let's examine how the extraction process works without defining any graph schema.

In [100]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
import os

# llm = ChatGoogleGenerativeAI(temperature=0, model='gemini-2.5-flash')
# llm = ChatOpenAI(temperature=0, model='gpt-4o')
llm = ChatOllama(temperature=0, model="nemotron-3-nano:30b")
# llm = ChatOllama(temperature=0, model="gpt-oss:20b")

In [101]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

no_schema = LLMGraphTransformer(llm=llm)

Now we can process the documents using the aconvert_to_graph_documents function, which is asynchronous. Using async with LLM extraction is recommended, as it allows for parallel processing of multiple documents. This approach can significantly reduce wait times and improve throughput, especially when dealing with multiple documents.

In [102]:
data = await no_schema.aconvert_to_graph_documents(documents)
print(data)

[GraphDocument(nodes=[Node(id='Albert Einstein', type='Person', properties={}), Node(id='Wikipedia:Gesprochener Artikel', type='Category', properties={}), Node(id='Wikipedia:Lesenswert', type='Category', properties={}), Node(id='Mitglied Der Familie Einstein', type='Relationship', properties={}), Node(id='Physiker (20. Jahrhundert)', type='Category', properties={}), Node(id='Astronom (20. Jahrhundert)', type='Category', properties={}), Node(id='Kosmologe (20. Jahrhundert)', type='Category', properties={}), Node(id='Nobelpreisträger Für Physik', type='Category', properties={}), Node(id='Träger Der Max-Planck-Medaille', type='Category', properties={}), Node(id='Träger Des Pour Le Mérite (Friedensklasse)', type='Category', properties={}), Node(id='Träger Der Copley-Medaille', type='Category', properties={}), Node(id='Ehrenbürger Von Tel Aviv-Jaffa', type='Category', properties={}), Node(id='Mitglied Der Leopoldina (20. Jahrhundert)', type='Relationship', properties={}), Node(id='Mitglied 

In [108]:
graph.add_graph_documents(data)

The response from the LLM Graph Transformer will be a graph document, which has the above structure. The graph document describes extracted nodes and relationships . Additionally, the source document of the extraction is added under the source key.

In [105]:
clean_graph()

Now, let's try the same extraction using the prompt-based approach. For models that support tools, you can enable prompt-based extraction by setting the `ignore_tool_usage` parameter.

In [8]:
no_schema_prompt = LLMGraphTransformer(llm=llm, ignore_tool_usage=True)
data = await no_schema_prompt.aconvert_to_graph_documents(documents)
graph.add_graph_documents(data)

In [9]:
clean_graph()

Next, let's walk through how defining a graph schema can help produce more consistent outputs.

## Defining allowed nodes
Constraining the extracted graph structure can be highly beneficial, as it guides the model to focus on specific, relevant entities and relationships. By defining a clear schema, you improve consistency across extractions, making the outputs more predictable and aligned with the information you actually need. This reduces variability between runs and ensures that the extracted data follows a standardized structure, capturing expected information. With a well-defined schema, the model is less likely to overlook key details or introduce unexpected elements, resulting in cleaner, more usable graphs.
We'll start by defining the expected types of nodes to extract using the allowed_nodesparameter.

In [10]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
nodes_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes)
data = await nodes_defined.aconvert_to_graph_documents(documents)
graph.add_graph_documents(data)

Here, we defined that the LLM should extract five types of nodes like Person, Organization, Location, and more.

In [11]:
clean_graph()

By specifying the expected node types, we achieve more consistent node extraction. However, some variation may still occur. For example, in the first run, "radioactivity" was extracted as a research field, while in the second, it was not.
Since we haven't defined relationships, their types can also vary across runs. Additionally, some extractions may capture more information than others. For instance, the `MARRIED_TO` relationship between Marie and Pierre isn't present in both extractions.
Now, let's explore how defining relationship types can further improve consistency.
## Defining allowed relationships
As we've observed, defining only node types still allows for variation in relationship extraction. To address this, let's explore how to define relationships as well. The first approach is to specify allowed relationships using a list of available types.

In [12]:
allowed_nodes = ["Person", "Organization", "Place", "Award", "ResearchField"]
allowed_relationships = [
    "SPOUSE",
    "AWARD",
    "FIELD_OF_RESEARCH",
    "WORKS_AT",
    "IN_LOCATION",
]
rels_defined = LLMGraphTransformer(
    llm=llm, allowed_nodes=allowed_nodes, allowed_relationships=allowed_relationships
)
data = await rels_defined.aconvert_to_graph_documents(documents)
graph.add_graph_documents(data)

With both nodes and relationships defined, our outputs become significantly more consistent. For example, Marie is always shown as winning an award, being the spouse of Pierre, and working at the University of Paris. However, since relationships are specified as a general list without restrictions on which nodes they can connect, some variation still occurs. For instance, the `FIELD_OF_RESEARCH` relationship might appear between a `Person` and a `ResearchField`, but sometimes it links an `Award` to a `ResearchField`. Additionally, since relationship directions aren't defined, there may be differences in directional consistency.
To address the issues of not being able to specify which nodes a relationship can connect and enforcing relationship direction, we recently introduced a new option for defining relationships, as shown below.

In [13]:
clean_graph()

Rather than defining relationships as a simple list of strings, we now use a three-element tuple format, where the elements represents the source node, relationship type, and target node, respectively.

In [14]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Person", "WORKS_AT", "Organization"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField"),
]
rels_defined = LLMGraphTransformer(
    llm=llm, allowed_nodes=allowed_nodes, allowed_relationships=allowed_relationships
)
data = await rels_defined.aconvert_to_graph_documents(documents)

Using the three-tuple approach provides a much more consistent schema for the extracted graph across multiple executions. However, given the nature of LLMs, there may still be some variation in the level of detail extracted. For instance, on the right side, Pierre is shown as winning the Nobel Prize, while on the left, this information is missing.
## Defining properties
The final enhancement we can make to the graph schema is to define properties for nodes and relationships. Here, we have two options. The first is setting either `node_properties` or `relationship_properties` to `true` allows the LLM to autonomously decide which properties to extract.

In [15]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Person", "WORKS_AT", "Organization"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField"),
]
node_properties = True
relationship_properties = True
props_defined = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=allowed_nodes,
    allowed_relationships=allowed_relationships,
    node_properties=node_properties,
    relationship_properties=relationship_properties,
)
data = await props_defined.aconvert_to_graph_documents(documents)
graph.add_graph_documents(data)

In [16]:
clean_graph()

We've enabled the LLM to add any node or relationship properties it considers relevant. For instance, it chose to include Marie Curie's birth and death dates, her role as a professor at the University of Paris, and the fact that she won the Nobel Prize twice. These additional properties significantly enrich the extracted information.

The second option we have is to define the node and relationship properties we want to extract.

In [17]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Person", "WORKS_AT", "Organization"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField"),
]
node_properties = ["birth_date", "death_date"]
relationship_properties = ["start_date"]
props_defined = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=allowed_nodes,
    allowed_relationships=allowed_relationships,
    node_properties=node_properties,
    relationship_properties=relationship_properties,
)
data = await props_defined.aconvert_to_graph_documents(documents)
graph.add_graph_documents(data)

In [18]:
clean_graph()

The birth and death dates remain consistent with the previous extraction. However, this time, the LLM also extracted the start date of Marie's professorship at the University of Paris.
Properties indeed add valuable depth to the extracted information, though there are currently some limitations in this implementation:
Properties can only be extracted using the tool-based approach.
All properties are extracted as strings.
Properties can only be defined globally, not per node label or relationship type.
There is no option to customize property descriptions to guide the LLM for more precise extraction.

## Strict mode
If you thought we had perfected a way to make the LLM follow the defined schema flawlessly, I have to set the record straight. While we invested considerable effort into prompt engineering, it's challenging to get LLM, especially the less performant one, to adhere to instructions with complete accuracy. To tackle this, we introduced a post-processing step, called strict_mode, that removes any information not conforming to the defined graph schema, ensuring cleaner and more consistent results.
By default, `strict_mode` is set to `True`, but you can disable it with the following code:
```
LLMGraphTransformer(
  llm=llm,
  allowed_nodes=allowed_nodes,
  allowed_relationships=allowed_relationships,
  strict_mode=False
)
```
With strict mode turned off, you may get node or relationship types outside the defined graph schema, as LLMs can sometimes take creative liberties with output structure.
## Importing graph documents into graph database
The extracted graph documents from the LLM Graph Transformer can be imported into graph databases like Neo4j for further analysis and applications using the add_graph_documents method. We'll explore different options for importing this data to suit different use cases.
### Default import
You can import nodes and relationships into Neo4j using the following code.

In [19]:
graph.add_graph_documents(data)

This method straightforwardly imports all nodes and relationships from the provided graph documents. We've used this approach throughout the blog post to review the results of different LLM and schema configurations.

In [20]:
clean_graph()

### Base entity label
Most graph databases support indexes to optimize data import and retrieval. In Neo4j, indexes can only be set for specific node labels. Since we might not know all the node labels in advance, we can handle this by adding a secondary base label to each node using the `baseEntityLabel` parameter. This way, we can still leverage indexing for efficient importing and retrieval without needing an index for every possible node label in the graph.

In [21]:
graph.add_graph_documents(data, baseEntityLabel=True)

As mentioned, using the baseEntityLabel parameter will result in each node having an additional __Entity__ label.

In [22]:
clean_graph()

## Include source documents
The final option is to also import the source documents for the extracted nodes and relationships. This approach lets us track which documents each entity appeared in. You can import the source documents using the include_source parameter.

In [23]:
graph.add_graph_documents(data, include_source=True)

In this visualization, the source document is highlighted in blue, with all entities extracted from it connected by `MENTIONS` relationships. This mode allows you to build retrievers that utilize both structured and unstructured search approaches.